In [1]:
import pandas as pd

In [4]:
data_import = pd.read_excel(f"../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/2023_2024/Skill Corner/data_physical.xlsx", index_col = 0)     


In [7]:
liste_dico = [{"comp_id" : 549,
                "ranking" : ["AJ Auxerre", "Angers SCO", "AS Saint-Étienne", "Rodez Aveyron", "Paris FC", "SM Caen", "Stade Lavallois Mayenne FC",
           "Amiens Sporting Club", "En Avant de Guingamp", "Pau FC", "Grenoble Foot 38", "Girondins de Bordeaux", "SC Bastia",
           "FC Annecy", "AC Ajaccio", "Dunkerque", "ES Troyes AC", "US Quevilly-Rouen", "US Concarneau", "Valenciennes FC"],
           "annee" : "2023_2024"},
           {"comp_id" : 393,
                "ranking" : ["Le Havre AC", "FC Metz", "Girondins de Bordeaux", "SC Bastia", "SM Caen", "En Avant de Guingamp", "Paris FC",
           "AS Saint-Étienne", "FC Sochaux-Montbéliard", "Grenoble Foot 38", "US Quevilly-Rouen", "Amiens Sporting Club", "Pau FC",
           "Rodez Aveyron", "Stade Lavallois Mayenne FC", "Valenciennes FC", "FC Annecy", "Dijon FCO", "Nîmes Olympique", "Chamois Niortais FC"],
           "annee" : "2022_2023"},
           {"comp_id" : 243,
                "ranking" : ["Toulouse FC", "AC Ajaccio", "AJ Auxerre", "Paris FC", "FC Sochaux-Montbéliard", "En Avant de Guingamp",
                             "SM Caen", "Le Havre AC", "Nîmes Olympique", "Pau FC", "Dijon FCO", "SC Bastia", "Chamois Niortais FC", 
                             "Amiens Sporting Club", "Grenoble Foot 38", "Valenciennes FC", "Rodez Aveyron", "US Quevilly-Rouen",
                             "Dunkerque", "AS Nancy-Lorraine"],
           "annee" : "2021_2022"}
           ]
dico = liste_dico[0]

In [35]:
data = data_import.set_index("team_name").fillna(0)

nb_matchs = pd.Series(index = data.index.unique())
for team in nb_matchs.index :
    nb_matchs[team] = len(data.loc[team].match_id.unique())
nb_matchs = nb_matchs.reindex(dico["ranking"])

drop = ["player_name", "player_short_name", 'psv99', "player_id", "player_birthdate", "team_id", "match_name", "match_date", "competition_name", "competition_id", "season_name",
    "season_id", "competition_edition_id", "position", "position_group", "minutes_full_tip", "minutes_full_otip", "physical_check_passed"]

data.drop(drop, inplace = True, axis = 1)

data.rename(columns = rename_col, inplace = True)

data = data.groupby(["team_name", "match_id"]).sum()

nb_minute_match = data.pop("minutes_per_Match")
data = data.multiply(900/nb_minute_match, axis = 0)

data = data.reset_index().drop("match_id", axis = 1).groupby("team_name", as_index = True).sum().reindex(dico["ranking"])

data = data.divide(nb_matchs, axis = 0)

data

,total_distance_per_Match,total_metersperminute_per_Match,running_distance_per_Match,hsr_distance_per_Match,hsr_count_per_Match,sprint_distance_per_Match,sprint_count_per_Match,hi_distance_per_Match,hi_count_per_Match,medaccel_count_per_Match,...,hsr_distance_per30otip,hsr_count_per30otip,sprint_distance_per30otip,sprint_count_per30otip,hi_distance_per30otip,hi_count_per30otip,medaccel_count_per30otip,highaccel_count_per30otip,meddecel_count_per30otip,highdecel_count_per30otip
team_name,,,,,,,,,,,,,,,,,,,,,
AJ Auxerre,100813.639007,1463.828358,13518.234328,5245.535322,514.998747,1655.009819,93.661388,6900.545141,608.660135,1062.879010,...,4335.918887,401.786332,1262.413612,61.570008,5598.331204,463.354773,625.793118,32.981554,440.838752,86.238147
Angers SCO,101557.788418,1494.205288,14295.595147,5672.401924,533.903232,1722.074322,98.418357,7394.476246,632.321589,1060.166973,...,4824.081847,427.900492,1238.257012,63.492920,6062.336720,491.401326,659.858122,33.279055,474.942963,89.740843
AS Saint-Étienne,102712.906449,1547.220231,14832.846226,5778.402988,546.933172,1677.032464,98.604791,7455.435452,645.537964,1069.439740,...,4942.785932,444.954643,1336.771949,65.144542,6279.561518,510.102258,677.980580,32.404489,467.239878,88.407690
Rodez Aveyron,103564.370403,1551.521839,14512.912063,5792.269769,551.936894,1889.752319,106.086177,7682.022088,658.023071,1081.202309,...,4800.763295,425.179844,1365.082136,66.521536,6165.848386,491.701628,663.556683,35.114329,471.156841,84.425596
Paris FC,103023.274224,1487.492067,14602.808283,5541.180702,528.701181,1663.620985,94.508366,7204.801687,623.209546,1114.444199,...,4701.103617,421.638318,1234.218500,60.708952,5935.320300,482.344699,663.295136,34.530344,463.720240,88.736262
SM Caen,102444.295369,1503.891627,14519.549050,5689.780044,550.320824,1765.811114,101.431146,7455.591157,651.751970,1089.613196,...,4647.264430,426.807927,1224.771006,61.856508,5872.034417,488.662859,663.582629,33.095224,473.344545,88.193068
Stade Lavallois Mayenne FC,103353.703711,1564.693382,14575.932991,5361.061407,498.333015,1515.057996,87.626726,6876.119404,585.959741,1131.806459,...,4331.978593,386.295710,1107.988506,54.789959,5439.970408,441.086190,684.521112,33.904253,497.354608,85.879217
Amiens Sporting Club,100979.516114,1452.863418,14393.521989,5554.438446,529.780950,1740.863257,98.691636,7295.301703,628.472586,1107.766203,...,4295.184350,396.324659,1117.270469,57.608317,5412.458125,453.938430,672.340628,34.169657,473.552439,90.145993
En Avant de Guingamp,104391.668809,1517.862028,14946.623110,5684.259221,548.016635,1722.830295,102.606897,7407.089516,650.623532,1131.976904,...,4827.197083,432.402161,1290.522085,67.163843,6117.718626,499.563292,673.850863,35.597402,475.828536,90.423695


In [20]:
data_col = data.columns[["full_otip_" in i for i in data.columns]]

In [29]:
def rename_col(col) :
    if "full_all" in col :
        return col.replace("full_all", "per_Match")
    elif "full_otip_p30otip" in col :
        return col.replace("full_otip_p30otip", "per30otip")
    elif "full_otip_p30tip" in col :
        return col.replace("full_otip_p30tip", "per30tip")
    else :
        return col

In [30]:
data.rename(columns = rename_col)

,match_id,minutes_per_Match,total_distance_per_Match,total_metersperminute_per_Match,running_distance_per_Match,hsr_distance_per_Match,hsr_count_per_Match,sprint_distance_per_Match,sprint_count_per_Match,hi_distance_per_Match,...,hsr_distance_per30otip,hsr_count_per30otip,sprint_distance_per30otip,sprint_count_per30otip,hi_distance_per30otip,hi_count_per30otip,medaccel_count_per30otip,highaccel_count_per30otip,meddecel_count_per30otip,highdecel_count_per30otip
team_name,,,,,,,,,,,,,,,,,,,,,
Angers SCO,1007272,97.37,9583.0,98.42,1230.0,355.0,39.0,145.0,8.0,500.0,...,378.03,34.89,79.97,4.36,458.00,39.26,65.43,4.36,40.71,7.27
Angers SCO,1021419,98.10,10355.0,105.56,1074.0,241.0,23.0,90.0,5.0,331.0,...,142.41,14.90,77.83,4.97,220.24,19.87,51.33,3.31,33.12,3.31
Angers SCO,1035154,102.35,10006.0,97.76,901.0,289.0,33.0,83.0,7.0,372.0,...,198.38,19.05,32.50,3.36,230.88,22.42,49.32,1.12,32.50,2.24
Angers SCO,1050074,99.27,10528.0,106.06,1054.0,273.0,27.0,60.0,4.0,333.0,...,231.11,20.00,36.67,1.11,267.78,21.11,54.44,2.22,35.56,4.44
Angers SCO,1064337,99.68,9441.0,94.71,876.0,186.0,22.0,34.0,3.0,220.0,...,149.13,14.60,16.69,2.09,165.82,16.69,51.10,2.09,33.37,6.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Girondins de Bordeaux,1543780,26.77,3076.0,114.92,436.0,139.0,19.0,57.0,3.0,196.0,...,364.93,54.25,152.88,4.93,517.81,59.18,49.32,0.00,39.45,9.86
Girondins de Bordeaux,1527200,10.67,1261.0,118.22,187.0,123.0,6.0,19.0,1.0,142.0,...,659.73,24.43,65.16,0.00,724.89,24.43,40.72,0.00,24.43,0.00
ES Troyes AC,1539379,12.60,1556.0,123.49,204.0,142.0,12.0,15.0,2.0,157.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [21]:
[j.replace("full_otip_", "") for j in data_col]

['total_distance_p30otip',
 'total_metersperminute_p30otip',
 'running_distance_p30otip',
 'hsr_distance_p30otip',
 'hsr_count_p30otip',
 'sprint_distance_p30otip',
 'sprint_count_p30otip',
 'hi_distance_p30otip',
 'hi_count_p30otip',
 'medaccel_count_p30otip',
 'highaccel_count_p30otip',
 'meddecel_count_p30otip',
 'highdecel_count_p30otip']

In [22]:
data.columns[["full_otip_" in i for i in data.columns]] = [j.replace("full_otip_", "") for j in data_col]

TypeError: Index does not support mutable operations